In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.dates as mdates
import mplcursors
import numpy as np
import matplotlib.cm as cm
%matplotlib notebook
plt.ion()
from sklearn.cluster import KMeans
import seaborn as sns
import plotly.graph_objects as go
from plotly import tools
from plotly.subplots import make_subplots
import plotly.offline as py

# Configure display options
pd.set_option('display.max_rows', None)     # To display all rows
pd.set_option('display.max_columns', None)  # To display all columns

df = pd.read_csv('B737MAX-8-M-03.csv')
sim=df['Sensor Name'][0]

#df.drop([:-4], axis=0, inplace=True)

if sim=='B737MAX-8-M-03':
    df=df[:-4]
else:
    df=df

  #new columns names based 'Raw Data' order
col_name=['phase1_avg','phase1_max','phase1_min','phase1_duty','phase2_avg','phase2_max','phase2_min','phase2_duty','phase3_avg','phase3_max','phase3_min','phase3_duty','value_Ah','sensor_state']

#split and convert it into list
split_values = df['Raw Data'].str.split('|').tolist()

# Create new columns in the DataFrame
for i, col_values in enumerate(zip(*split_values)):
    df[col_name[i]] = [float(val) for val in col_values]

    
df = df.rename(columns={'Date': 'Timestamp' }) 
df[['Date', 'Time']] = df['Timestamp'].str.split(' ', 1, expand=True)    
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df['Date'] = pd.to_datetime(df['Date'])
# Extract the number of seconds since midnight from the 'Time' column
df['Time'] = pd.to_datetime(df['Time']).dt.hour * 3600 + \
             pd.to_datetime(df['Time']).dt.minute * 60 #+ \
             #pd.to_datetime(df['Time']).dt.second
    
# Display the updated DataFrame
df.head()


df.set_index('Timestamp', inplace=True)
weekly_consumption = df['Value'].resample('W').mean()

# Plot the weekly consumption trends
plt.figure(figsize=(10, 6))
weekly_consumption.plot(marker='o')
plt.xlabel('Week')
plt.ylabel('Average Consumption')
plt.title('Weekly Consumption Trends')
image_name = f"{sim}_weekly Current Consumption.png"
plt.savefig (image_name)
plt.grid(True)
plt.show()

# Calculate daily consumption
daily_consumption = df['phase1_avg']

# Create the histogram
plt.figure(figsize=(10, 6), dpi=100)
plt.hist(daily_consumption, bins=20, edgecolor='black')
#plt.xticks(range(0,250, 10))
plt.xlabel('Daily Consumption (Ah)- Phase 1')
plt.ylabel('Frequency')
plt.title('Histogram of Daily Consumption')
plt.grid(True)
image_name = f"{sim}_Histogram of Daily Consumption.png"
plt.savefig (image_name)
plt.show()

# Create a 3D plot
fig = plt.figure(figsize=(5, 3), dpi=300)
ax = fig.add_subplot(111, projection='3d')


ax.set_xlabel('Date')
ax.set_ylabel('Time (hrs)')
ax.set_zlabel('Current Consumption (A)')
x = mdates.date2num(df['Date'])
ax.set_zlim(0, 25)
ax.set_ylim(0, 24)  # Change the range to 0-24 hours
# Plot the 3D scatter plot
#ax.scatter(x, df['Time']/3600, df['phase1_avg'])
phase1=ax.scatter(x, df['Time']/3600, df['phase1_avg'], label='Phase 1')
phase2=ax.scatter(x, df['Time']/3600, df['phase2_avg'], label='Phase 2')
phase3=ax.scatter(x, df['Time']/3600, df['phase3_avg'], label='Phase 3')

# Format the 'Date' axis with date labels
date_formatter = mdates.DateFormatter('%Y-%m-%d')
ax.xaxis.set_major_formatter(date_formatter)
fig.autofmt_xdate()
ax.tick_params(axis='x', labelsize=6)
ax.tick_params(axis='y', labelsize=6)
ax.tick_params(axis='z', labelsize=6)
ax.legend(handles=[phase1, phase2, phase3])

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.tight_layout()
image_name = f"{sim}_3D plot of total Current Consumption.png"
plt.savefig (image_name)
# Show the plot
plt.show()

df.plot(y=['phase1_avg', 'phase2_avg', 'phase3_avg'], figsize=(10, 6))

# Set the title and labels
plt.title('Three Phases')
plt.xlabel('Date')
plt.ylabel('Value (A)')

plt.tight_layout()

#plt.savefig('Three Phases.png', dpi=800)
image_name = f"{sim}_Three phase plot.png"
plt.savefig (image_name)
# Display the chart
plt.show()
# Iterate over the unique dates and plot the hourly averages
lines = []
for date in df_avg['Date'].unique():
    data = df_avg[df_avg['Date'] == date]
    line, = ax.plot(data['Hour'], data['Average'], label=str(date))
    lines.append(line)

# Create hover events for each line using mplcursors
mplcursors.cursor(lines).connect("add", lambda sel: sel.annotation.set_text(sel.artist.get_label()))

# Set the labels and title
ax.set_xlabel('Hour')
ax.set_ylabel('Value (Ah)')
ax.set_title('Daily Hourly Averages Total RMS Current')
ax.set_xticks(range(25))

# Add a legend
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=7)

# Set the same limits for both x and y axes
ax.set_xlim(0, 24)
ax.set_ylim(0, 10)  # Adjust the limits based on your data

plt.tight_layout()
image_name = f"{sim}_Daily Hourly Average Total RMS Current.png"
plt.savefig (image_name)
# Show the plot
plt.show()

import seaborn as sns
df_hourly = df.resample('H').mean()
#plt.figure(figsize=(12, 6))  # Adjust the figure size as needed

df_pivot = df_hourly.pivot_table(index=df_hourly.index.date, columns=df_hourly.index.hour, values='Value')

# Create a heatmap using seaborn
plt.figure(figsize=(12, 6))  # Adjust the figure size as needed
sns.heatmap(df_pivot, cmap='inferno')

# Set labels and title
plt.xlabel('Hour of the Day')
plt.ylabel('Date')
plt.title('Current Consumption Heatmap')
plt.tight_layout()
image_name = f"{sim}_current consumption Heatmap.png"
plt.savefig (image_name)

# Display the heatmap
plt.show()


